In [1]:
import pandas as pd
import glob
import pysam
import os
from collections import defaultdict
import sys
from get_pairs import *

directory = '/scratch/groups/dpwall/personal/chloehe/unmapped_reads/bam'
pd.set_option('display.max_rows', 100)

## Fam 1

In [28]:
# def pairs_generator(file, refname=None):
    
#     read_dict = defaultdict(lambda: [None, None])
#     file.reset()
#     for read in file.fetch(refname):
#         qname = read.query_name
#         if qname in read_dict:
#             if read.is_read1:
#                 yield qname, read, read_dict[qname][1]
#             else:
#                 yield qname, read_dict[qname][0], read
#             del read_dict[qname]
#         else:
#             if read.is_read1:
#                 read_dict[qname][0] = read
#             else:
#                 read_dict[qname][1] = read

In [2]:
def get_pairs_from_pairend_alignment(directory, refname=None):
    
    # check for alignment file and index file
    try:
        filename = glob.glob(os.path.join(directory, '*final.paired.aln_all.bam'))[0]
    except:
        print('Paired end alignment file not found!\nExiting from get_pairs_from_pairend_alignment')
        exit()
    if not os.path.isfile(filename+'.bai'):
        pysam.samtools.index(filename)
    else:
        if os.path.getctime(filename+'.bai') < os.path.getctime(filename):
            pysam.samtools.index(filename)
    
    output_dict = dict()
    read_dict = defaultdict(lambda: [None, None])
    
    alignmentfile = pysam.AlignmentFile(filename, 'rb')
    alignmentfile.reset()
    for read in alignmentfile.fetch(refname, until_eof=True):
        qname = read.query_name
        if qname in read_dict:
            # sanity check: no qname should have been in output_dict already
            if qname in output_dict:
                print('Duplicate read warning: {}, skip and continue'.format(qname))
                continue
            if read.is_unmapped:
                if read.is_read1:
                    r2 = read_dict[qname][1]
                    if r2 is 'unmapped':
                        output_dict[qname] = ['unmapped', 'unmapped', 'N/A',
                                              r2, r2, 'N/A',
                                              'False']
                    else:
                        output_dict[qname] = ['unmapped', 'unmapped', 'N/A',
                                              alignmentfile.get_reference_name(r2.reference_id),
                                              r2.reference_start,
                                              r2.mapping_quality,
                                              'False']
                    
                else:
                    r1 = read_dict[qname][0]
                    if r1 is 'unmapped':
                        output_dict[qname] = [r1, r1, 'N/A',
                                              'unmapped', 'unmapped', 'N/A',
                                              'False']
                    else:
                        output_dict[qname] = [alignmentfile.get_reference_name(r1.reference_id),
                                              r1.reference_start,
                                              r1.mapping_quality,
                                              'unmapped', 'unmapped', 'N/A',
                                              'False']
            else:
                if read.is_read1:
                    r2 = read_dict[qname][1]
                    if r2 is 'unmapped':
                        output_dict[qname] = [alignmentfile.get_reference_name(read.reference_id),
                                              read.reference_start,
                                              read.mapping_quality,
                                              r2, r2, 'N/A',
                                              'False']
                    else:
                        output_dict[qname] = [alignmentfile.get_reference_name(read.reference_id),
                                              read.reference_start,
                                              read.mapping_quality,
                                              alignmentfile.get_reference_name(r2.reference_id),
                                              r2.reference_start,
                                              r2.mapping_quality,
                                              read.is_proper_pair]
                    
                else:
                    r1 = read_dict[qname][0]
                    if r1 is 'unmapped':
                        output_dict[qname] = [r1, r1, 'N/A',
                                              alignmentfile.get_reference_name(read.reference_id),
                                              read.reference_start,
                                              read.mapping_quality,
                                              'False']
                    else:
                        output_dict[qname] = [alignmentfile.get_reference_name(r1.reference_id),
                                              r1.reference_start,
                                              r1.mapping_quality,
                                              alignmentfile.get_reference_name(read.reference_id),
                                              read.reference_start,
                                              read.mapping_quality,
                                              r1.is_proper_pair]
            # delete entry from dictionary as soon as its mate is found
            del read_dict[qname]
        else:
            if read.is_unmapped:
                if read.is_read1:
                    read_dict[qname][0] = 'unmapped'
                else:
                    read_dict[qname][1] = 'unmapped'
            else:
                if read.is_read1:
                    read_dict[qname][0] = read
                else:
                    read_dict[qname][1] = read

    # sanity check: read_dict should be empty if all single-end reads found their mates
    if bool(read_dict):
        print('Warning: pairing error!')
        for qname in read_dict:
            print('Mate is not found for '+qname)
        print('Skip and continue')
        
    alignmentfile.close()
    print('Done processing paired-end alignment file!')
    
    return output_dict

In [12]:
paired_alignment_dict = get_pairs_from_pairend_alignment(os.path.join(directory, 'fam1/MH0143018'))

Done processing paired-end alignment file!


In [13]:
paired_df = pd.DataFrame.from_dict(paired_alignment_dict, orient='index')
paired_df.columns = ['R1_ref', 'R1_start', 'R1_MAPQ', 'R2_ref', 'R2_start', 'R2_MAPQ', 'is_proper_pair']
paired_df

,R1_ref,R1_start,R1_MAPQ,R2_ref,R2_start,R2_MAPQ,is_proper_pair
E00218:136:HC35MCCXX:4:2117:4400:26062,MH533023.1,316,60,MH533023.1,38,60,True
E00218:136:HC35MCCXX:6:1105:16083:63420,MH533023.1,325,50,MH533023.1,86,40,True
E00218:136:HC35MCCXX:6:1116:6257:25025,MH533023.1,432,60,MH533023.1,195,60,True
E00218:136:HC35MCCXX:2:1201:2299:47352,MH533023.1,43,60,MH533023.1,438,60,True
E00218:136:HC35MCCXX:4:1213:2238:67780,MH533023.1,454,60,MH533023.1,254,60,True
...,...,...,...,...,...,...,...
E00218:136:HC35MCCXX:8:2224:32505:25939,unmapped,unmapped,N/A,unmapped,unmapped,N/A,False
E00218:136:HC35MCCXX:8:2224:32526:4596,unmapped,unmapped,N/A,unmapped,unmapped,N/A,False
E00218:136:HC35MCCXX:8:2224:32546:28787,unmapped,unmapped,N/A,unmapped,unmapped,N/A,False
E00218:136:HC35MCCXX:8:2224:32627:67006,unmapped,unmapped,N/A,unmapped,unmapped,N/A,False


In [3]:
def get_pairs_from_singleend_alignment(directory, refname=None):
    
   # check for alignment + original files and index files
    try:
        files = [glob.glob(os.path.join(directory, '*final.single.aln_all.bam'))[0],
                 glob.glob(os.path.join(directory, '*final.improper_highq.bam'))[0],
                 glob.glob(os.path.join(directory, '*final.map_unmap.bam'))[0]]

    except:
        print('One or more missing files!\nExiting from get_pairs_from_singleend_alignment')
        exit()
    for file in files:
        if not os.path.isfile(file+'.bai'):
            pysam.samtools.index(file)
        else:
            if os.path.getctime(file+'.bai') < os.path.getctime(file):
                pysam.samtools.index(file)
    
    output_dict = dict()
    read_dict = defaultdict(lambda: [None, None])
    
    alignmentfile = pysam.AlignmentFile(files[0], 'rb')
    duplicates = 0
    for read in alignmentfile.fetch(refname, until_eof=True):
        qname = read.query_name
        # sanity check: no qname should have been in output_dict already
        if qname in read_dict:
            print('Duplicate read warning! {} is a duplicate'.format(qname))
            duplicates += 1
            continue
        # single-end reads don't have read1/read2 tags
        if read.is_unmapped:
            read_dict[qname][0] = 'unmapped'
        else:
            read_dict[qname][0] = read
    if duplicates > 0:
        print('{} duplicate reads found in single-end alignment file. Skip and continue'.format(duplicates))

    print('{} reads retrieved from single-end realignment'.format(len(read_dict)))
    originalfiles = []
    for file in files[1:]:
        originalfile = pysam.AlignmentFile(file, 'rb')
        originalfiles.append(originalfile)
        for read in originalfile.fetch(refname, until_eof=True):
            qname = read.query_name
            if qname in read_dict:
                if read.is_read1 and read_dict[qname][0] is not None:
                    r2 = read_dict[qname][0]
                    if read.is_unmapped:
                        if r2 is 'unmapped':
                            output_dict[qname] = ['unmapped', 'unmapped', 'N/A',
                                                  r2, r2, 'N/A',
                                                  'False']
                        else:
                            output_dict[qname] = ['unmapped', 'unmapped', 'N/A',
                                                  alignmentfile.get_reference_name(r2.reference_id),
                                                  r2.reference_start,
                                                  r2.mapping_quality,
                                                  'False']
                    else:
                        if r2 is 'unmapped':
                            output_dict[qname] = [originalfile.get_reference_name(read.reference_id),
                                                  read.reference_start,
                                                  read.mapping_quality,
                                                  r2, r2, 'N/A',
                                                  'False']
                        else:
                            output_dict[qname] = [originalfile.get_reference_name(read.reference_id),
                                                  read.reference_start,
                                                  read.mapping_quality,
                                                  alignmentfile.get_reference_name(r2.reference_id),
                                                  r2.reference_start,
                                                  r2.mapping_quality,
                                                  read.is_proper_pair]
                    # delete entry from dictionary as soon as its mate is found
                    del read_dict[qname]
                elif read.is_read2 and read_dict[qname][0] is not None:
                    r1 = read_dict[qname][0]
                    if read.is_unmapped:
                        if r1 is 'unmapped':
                            output_dict[qname] = [r1, r1, 'N/A',
                                                  'unmapped', 'unmapped', 'N/A',
                                                  'False']
                        else:
                            output_dict[qname] = [alignmentfile.get_reference_name(r1.reference_id), 
                                                  r1.reference_start,
                                                  r1.mapping_quality,
                                                  'unmapped', 'unmapped', 'N/A',
                                                  'False']
                    else:
                        if r1 is 'low MAPQ' or r1 is 'unmapped':
                            output_dict[qname] = [r1, r1, 'N/A',
                                                  originalfile.get_reference_name(read.reference_id),
                                                  read.reference_start,
                                                  read.mapping_quality,
                                                  'False']
                        else:
                            output_dict[qname] = [alignmentfile.get_reference_name(r1.reference_id), 
                                                  r1.reference_start,
                                                  r1.mapping_quality,
                                                  originalfile.get_reference_name(read.reference_id),
                                                  read.reference_start,
                                                  read.mapping_quality,
                                                  r1.is_proper_pair]
                    # delete entry from dictionary as soon as its mate is found
                    del read_dict[qname]
            # should only execute else when file == improper_highq.bam
            else:
                if 'map_unmap' in file:
                    print('Warning: {} not found in realignment'.format(qname))
                    
                # if file == improper_highq.bam, also retrieve reads not found in read_dict
                # as improperly paired high score reads
                else:
                    read_dict[qname][0] = read
        originalfile.close()        

    # sanity check: read_dict should be empty if all single-end reads found their mates
    if bool(read_dict):
        print('Pairing error! {} single-end reads do not have mates'.format(len(read_dict)))
        for qname in read_dict:
            print('Mate is not found for {}. Skip and continue'.format(qname))
    
    alignmentfile.close()
    print('Done processing single-end alignment file!')
    
    return output_dict

In [25]:
single_alignment_dict = get_pairs_from_singleend_alignment(os.path.join(directory, 'fam1/MH0143018'))

4076492 single-end reads found
Done processing single-end alignment file!


In [26]:
single_df = pd.DataFrame.from_dict(single_alignment_dict, orient='index')
single_df.columns = ['R1_ref', 'R1_start', 'R1_MAPQ', 'R2_ref', 'R2_start', 'R2_MAPQ', 'is_proper_pair']
single_df

,R1_ref,R1_start,R1_MAPQ,R2_ref,R2_start,R2_MAPQ,is_proper_pair
E00218:136:HC35MCCXX:1:1217:17930:70980,chr1,10000,0,unmapped,unmapped,N/A,False
E00218:136:HC35MCCXX:4:1110:14296:56599,chr1,10000,0,unmapped,unmapped,N/A,False
E00218:136:HC35MCCXX:4:2112:26902:27802,chr1,10000,0,unmapped,unmapped,N/A,False
E00218:136:HC35MCCXX:5:2120:16580:38122,chr1,10000,1,unmapped,unmapped,N/A,False
E00218:136:HC35MCCXX:6:2106:13393:36786,chr1,10000,0,unmapped,unmapped,N/A,False
...,...,...,...,...,...,...,...
E00218:136:HC35MCCXX:1:1222:3710:30492,HLA-DQB1*03:03:02:02,644,0,unmapped,unmapped,N/A,False
E00218:136:HC35MCCXX:3:2121:31308:37542,HLA-DQB1*03:05:01,5681,0,unmapped,unmapped,N/A,False
E00218:136:HC35MCCXX:4:1202:30313:4948,HLA-DRB1*04:03:01,6848,0,unmapped,unmapped,N/A,False
E00218:136:HC35MCCXX:7:2224:4481:6407,HLA-DRB1*08:03:02,5855,0,unmapped,unmapped,N/A,False


In [4]:
def generate_alignment_table(directory, mapq_threshold=10, refname=None):
    
    # create dictionary for pairs from realignment
    alignment_dict = {**get_pairs_from_pairend_alignment(directory), **get_pairs_from_singleend_alignment(directory)}
    print('Total: {} pairs of reads found from realignment'.format(alignment_dict))
    
    # check new alignment dict against original alignments and 
    # replace unmapped/low score alignments with original alignment
    # check for alignment file and index file
    try:
        filename = glob.glob(os.path.join(directory, '*final.improper_lowq.bam'))[0]
    except:
        print('Alignment file not found!\nExiting from generate_alignment_table')
        exit()
    if not os.path.isfile(filename+'.bai'):
        pysam.samtools.index(filename)
    else:
        if os.path.getctime(filename+'.bai') < os.path.getctime(filename):
            pysam.samtools.index(filename)
    
    alignmentfile = pysam.AlignmentFile(filename, 'rb')
    count = 0
    for read in alignmentfile.fetch(refname):
        qname = read.query_name
        if qname in alignment_dict:
            if read.is_read1:
                if alignment_dict[qname][0] == 'unmapped' or alignment_dict[qname][2] < mapq_threshold:
                    alignment_dict[qname][:3] = [alignmentfile.get_reference_name(read.reference_id),
                                                 read.reference_start,
                                                 read.mapping_quality]
            else:
                if alignment_dict[qname][3] == 'unmapped' or alignment_dict[qname][5] < mapq_threshold:
                    alignment_dict[qname][3:6] = [alignmentfile.get_reference_name(read.reference_id),
                                                 read.reference_start,
                                                 read.mapping_quality]
        # sanity check: all reads should have been realigned and be found in alignment_dict
        else:
            count += 1
            print('Warning: read {} not found in realignment. Skip and continue'.format(qname))
        
    if count > 0:
        print('Warning: {} reads not found in realignment!')
            
    return alignment_dict

In [5]:
%%time
alignment_dict = generate_alignment_table(os.path.join(directory, 'fam1/MH0143018'))

Done processing paired-end alignment file!
4076492 single-end reads found from single-end realignment
Done processing single-end alignment file!


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


CPU times: user 4min 44s, sys: 1min 5s, total: 5min 49s
Wall time: 7min 1s


In [6]:
alignment_df = pd.DataFrame.from_dict(alignment_dict, orient='index')
alignment_df.columns = ['R1_ref', 'R1_start', 'R1_MAPQ', 'R2_ref', 'R2_start', 'R2_MAPQ', 'is_proper_pair']
alignment_df

,R1_ref,R1_start,R1_MAPQ,R2_ref,R2_start,R2_MAPQ,is_proper_pair
E00218:136:HC35MCCXX:4:2117:4400:26062,MH533023.1,316,60,MH533023.1,38,60,True
E00218:136:HC35MCCXX:6:1105:16083:63420,MH533023.1,325,50,MH533023.1,86,40,True
E00218:136:HC35MCCXX:6:1116:6257:25025,MH533023.1,432,60,MH533023.1,195,60,True
E00218:136:HC35MCCXX:2:1201:2299:47352,MH533023.1,43,60,MH533023.1,438,60,True
E00218:136:HC35MCCXX:4:1213:2238:67780,MH533023.1,454,60,MH533023.1,254,60,True
...,...,...,...,...,...,...,...
E00218:136:HC35MCCXX:1:1222:3710:30492,HLA-DQB1*03:03:02:02,644,0,unmapped,unmapped,N/A,False
E00218:136:HC35MCCXX:3:2121:31308:37542,HLA-DQB1*03:05:01,5681,0,unmapped,unmapped,N/A,False
E00218:136:HC35MCCXX:4:1202:30313:4948,HLA-DRB1*04:03:01,6848,0,unmapped,unmapped,N/A,False
E00218:136:HC35MCCXX:7:2224:4481:6407,HLA-DRB1*08:03:02,5855,0,unmapped,unmapped,N/A,False


In [7]:
alignment_df.to_csv(os.path.join(directory, 'fam1/MH0143018/MH0143018.final_alignment_table.csv'))

## Fam 3

In [2]:
paired_alignment_dict = get_pairs_from_pairend_alignment(os.path.join(directory, 'fam3/03C16794'))
paired_df = pd.DataFrame.from_dict(paired_alignment_dict, orient='index')
paired_df.columns = ['R1_ref', 'R1_start', 'R1_MAPQ', 'R2_ref', 'R2_start', 'R2_MAPQ', 'is_proper_pair']

Done processing paired-end alignment file!


In [3]:
single_alignment_dict = get_pairs_from_singleend_alignment(os.path.join(directory, 'fam3/03C16794'))
single_df = pd.DataFrame.from_dict(single_alignment_dict, orient='index')
single_df.columns = ['R1_ref', 'R1_start', 'R1_MAPQ', 'R2_ref', 'R2_start', 'R2_MAPQ', 'is_proper_pair']

3071869 reads retrieved from single-end realignment
Done processing single-end alignment file!


In [2]:
alignment_dict = generate_alignment_dict(os.path.join(directory, 'fam3/03C16794'))

Done processing paired-end alignment file!
3071869 reads retrieved from single-end realignment
Done processing single-end alignment file!
Total: 7760642 pairs of reads found from realignment


In [4]:
sampleid = '03C16794'

alignment_df = pd.DataFrame.from_dict(alignment_dict, orient='index')
alignment_df.columns = ['R1_ref', 'R1_start', 'R1_MAPQ', 'R2_ref', 'R2_start', 'R2_MAPQ', 'is_proper_pair']

alignment_df.to_csv(os.path.join(directory, 'fam3/03C16794', sampleid+'.final_alignment_table.csv'))